In [1]:
# CS124 Programming Assignment 3
# Christian Hallas
# Spring 2016

In [2]:
# Import packages
import heapq
import random

In [97]:
# Constants defined
NO_INTS = 100

In [98]:
def random_ints(num, lower=0, upper=10 ** 12):
    return [random.randrange(lower,upper+1) for i in range(num)]

In [145]:
def karmarkarkarp(A=[]):
    if not A:
        A = random_ints(NO_INTS)
        A_neg = [-x for x in A]
    heapq.heapify(A_neg)
    for _ in range(NO_INTS-1):
        max_int1 = heapq.heappop(A_neg)
        max_int2 = heapq.heappop(A_neg)
        heapq.heappush(A_neg, max_int1-max_int2)
    return -A_neg[0]

In [164]:
residues = []
for _ in range(10000):
    residues.append(karmarkarkarp())

In [165]:
sum(residues)/10000

255303